### Create Technical Threshold Rasters
In this notebook we create three technical threshold rasters (1) Slope threshold raster (2) Population threshold raster (3) Flood risk threshold raster. These rasters will be masked from the Technical Inclusion Area Layer to generate the SL1 Inclusion Area Layer. 

In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define helpful functions

In [2]:
def get_datum_transformation(source_spatial_reference, target_spatial_reference):
    transformations = arcpy.ListTransformations(source_spatial_reference, target_spatial_reference)
    if transformations:
        return transformations[0]  # Use the first transformation found
    else:
        return None  # No appropriate transformation found

In [3]:
def add_occupancy_field(feature_class):
    # Check if the 'occupancy' field already exists
    fields = arcpy.ListFields(feature_class, "occupancy")

    if not fields:
         # Add 'occupancy' field to the feature class if it doesn't exist
        arcpy.management.AddField(feature_class, "occupancy", "SHORT")

        # Set the 'occupancy' field value to 1 for all records
        with arcpy.da.UpdateCursor(feature_class, "occupancy") as cursor:
            for row in cursor:
                row[0] = 1
                cursor.updateRow(row)
        print(f"'occupancy' field added and set to 1 in {os.path.basename(feature_class)}")
    else:
        print(f"'occupancy' field already exists in {os.path.basename(feature_class)}")

In [5]:
def convert_to_one_bit(input_raster, output_raster):
    try:
        # Create a raster object from the input path
        input_raster_obj = arcpy.sa.Raster(input_raster)

        # Reclassify the input raster to binary (one-bit) format
        binary_raster = arcpy.sa.Con(input_raster_obj > 0, 1)

        # Save the binary raster to the specified output path
        binary_raster.save(output_raster)

        print(f"Conversion to one-bit raster completed successfully. Result saved to {output_raster}")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")


#### Get slope threshold raster

In [6]:
# Set input and output folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"  # enter path to input folder

In [7]:
# Set path to wind workspace
wind_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Wind")

if not os.path.exists(wind_workspace):
    os.makedir(wind_workspace)

# Set path to solar workspace
solar_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Solar")

if not os.path.exists(solar_workspace):
    os.makedir(solar_workspace)

In [8]:
# Set path to input DEM
input_dem = os.path.join(mainInputFolder, "SRTM_90m.tif")

# Set path to output projected DEM,resample DEM, and slope raster
projected_dem = os.path.join(mainInputFolder, os.path.splitext(os.path.basename(input_dem))[0] + "_proj.tif")
resampled_dem = os.path.join(mainInputFolder, "SRTM_250m_proj.tif")
slope_raster =  os.path.join(mainInputFolder, "SRTM_250m_slope.tif")

In [9]:
# get the input spatial reference
input_sr = arcpy.Describe(input_dem).spatialReference
input_sr

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [10]:
# Specify the output spatial reference
output_sr = arcpy.SpatialReference(102039)

# Specify the target cell size
cell_size = "250"

In [11]:
# Define datum transform
datum_transform = get_datum_transformation(input_sr, output_sr)
datum_transform

'WGS_1984_(ITRF00)_To_NAD_1983'

In [13]:
# Set overwrite mode to true
arcpy.env.overwriteOutput = True

try:
    # Project the DEM
    arcpy.ProjectRaster_management(in_raster = input_dem,
                                   out_raster = projected_dem,
                                   out_coor_system = output_sr,
                                   resampling_type = 'BILINEAR',
                                   geographic_transform = datum_transform)
    print("DEM reprojected successfully")

    # Resample the projected DEM to the target cell size
    arcpy.Resample_management(projected_dem, resampled_dem, cell_size, "BILINEAR")
    print("DEM resampled successfully")

    # Calculate the slope of the resampled DEM
    arcpy.Slope_3d(resampled_dem, slope_raster, "PERCENT_RISE")
    print("Slope calculation completed successfully!")

except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

DEM reprojected successfully
DEM resampled successfully
Slope calculation completed successfully!


In [56]:
# Reclassified slope raster (threshold raster) for wind
wind_slope_threshold = os.path.join(wind_workspace, os.path.splitext(os.path.basename(slope_raster))[0]+ "_wind.tif")
wind_slope_threshold

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\Wind\\SRTM_250m_slope_wind.tif'

In [81]:
# convert to binary raster
wind_slope_binary = os.path.join(wind_workspace, os.path.splitext(os.path.basename(wind_slope_threshold))[0]+ "_binary.tif")
wind_slope_binary

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\Wind\\SRTM_250m_slope_wind_binary.tif'

In [11]:
# Define the remap table
remap_table_wind = arcpy.sa.RemapRange([
    [0, 20, 0],
    [20, 2000, 1],
])

In [12]:
# Reclassify the slope raster using the remap table
reclassified_wind_slope = arcpy.sa.Reclassify(slope_raster, "VALUE", remap_table_wind)

# Save the reclassified raster to the output file path
reclassified_wind_slope.save(wind_slope_threshold)

In [55]:
# Reclassified slope raster (threshold raster) for wind
solar_slope_threshold = os.path.join(solar_workspace, os.path.splitext(os.path.basename(slope_raster))[0]+ "_solar.tif")
solar_slope_threshold

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\Solar\\SRTM_250m_slope_solar.tif'

In [82]:
# convert to binary raster
solar_slope_binary = os.path.join(solar_workspace, os.path.splitext(os.path.basename(solar_slope_threshold))[0]+ "_binary.tif")
solar_slope_binary

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\Solar\\SRTM_250m_slope_solar_binary.tif'

In [14]:
# Define the remap table
remap_table_solar = arcpy.sa.RemapRange([
    [0, 10, 0],
    [10, 2000, 1],
])

In [15]:
# Reclassify the slope raster using the remap table
reclassified_solar_slope = arcpy.sa.Reclassify(slope_raster, "VALUE", remap_table_solar)

# Save the reclassified raster to the output file path
reclassified_solar_slope.save(solar_slope_threshold)

In [85]:
# Convert to binary raster
convert_to_one_bit(solar_slope_threshold, solar_slope_binary)
convert_to_one_bit(wind_slope_threshold, wind_slope_binary)

Conversion to one-bit raster completed successfully. Result saved to C:\Users\Zachary\ASSET\supplyCurve\analysis\data\technicalExclusionRasters\Solar\SRTM_250m_slope_solar_binary.tif
Conversion to one-bit raster completed successfully. Result saved to C:\Users\Zachary\ASSET\supplyCurve\analysis\data\technicalExclusionRasters\Wind\SRTM_250m_slope_wind_binary.tif


In [10]:
# set environmental raster settings to the reclassified slope raster
arcpy.env.snapRaster = slope_raster
arcpy.env.extent = slope_raster
arcpy.env.cellSize = slope_raster

#### Get population threshold raster
*consider reclassifying before resampling here*

In [16]:
# Set path to input LANDSCAN raster
landscan_usa = os.path.join(mainInputFolder,"landscan-usa-2020-conus-day.tif")

In [36]:
# set paths to projected, resampled, and reclassified landscan raster
landscan_projected = os.path.join(mainInputFolder, "landscan_conus_2020_projected.tif")
landscan_resampled = os.path.join(mainInputFolder, "landscan_conus_2020_250m.tif")
landscan_threshold = os.path.join(mainInputFolder, "technicalExclusionRasters\\landscan_conus_2020_threshold_100.tif")


In [22]:
# Get spatial reference of landscan raster
landscan_sr = arcpy.Describe(landscan_usa).spatialReference
landscan_sr

name (Geographic Coordinate System),GCS_WGS_1984
factoryCode (WKID),4326
angularUnitName (Angular Unit),Degree
datumName (Datum),D_WGS_1984


In [23]:
# Get datum transform
landscan_datum_transform = get_datum_transformation(landscan_sr, output_sr)
landscan_datum_transform

'WGS_1984_(ITRF00)_To_NAD_1983'

In [24]:
try:
    # Project the LANDSCAN raster
    arcpy.ProjectRaster_management(in_raster = landscan_usa,
                                   out_raster = landscan_projected,
                                   out_coor_system = output_sr,
                                   resampling_type = 'BILINEAR',
                                   geographic_transform = datum_transform)
    print("LANDSCAN reprojected successfully")

    # Resample the projected LANDSCAN raster to the target cell size
    arcpy.Resample_management(landscan_projected, landscan_resampled, cell_size, "BILINEAR")
    print("LANDSCAN resampled successfully")
    
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

LANDSCAN reprojected successfully
LANDSCAN resampled successfully


In [25]:
# Define the remap table
landscan_remap = arcpy.sa.RemapRange([
[0,0.81, 0],
[0.81, 15500, 1]
])

In [27]:
# Reclassify the LANDSCAN raster using the remap table
landscan_reclassified = arcpy.sa.Reclassify(landscan_resampled, "VALUE", landscan_remap)

# Save the reclassified raster to the output file path
landscan_reclassified.save(landscan_threshold)

In [64]:
# Set path to binary raster 
landscan_binary = os.path.join(mainInputFolder, "technicalExclusionRasters\\landscan_conus_2020_threshold_100_binary.tif")
landscan_binary

'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\landscan_conus_2020_threshold_100_binary.tif'

In [86]:
# convert to binary raster
convert_to_one_bit(landscan_threshold, landscan_binary)

Conversion to one-bit raster completed successfully. Result saved to C:\Users\Zachary\ASSET\supplyCurve\analysis\data\technicalExclusionRasters\landscan_conus_2020_threshold_100_binary.tif


#### Get Flood Threshold Raster

In [11]:
# Set path to flood hazard layer
flood_hazard = os.path.join(mainInputFolder, "floodHazard.gdb\\FEMA_flood_hazard_zones")


In [12]:
# Set path to projected flood hazard layer and raster layer
flood_hazard_projected = os.path.join(mainInputFolder, "floodHazard.gdb\\FEMA_flood_hazard_zones_projected")
flood_hazard_raster = os.path.join(mainInputFolder, "technicalExclusionRasters\\FEMA_flood_hazard_zones.tif")

In [13]:
# Get flood hazard projection
flood_hazard_sr = arcpy.Describe(flood_hazard).spatialReference
flood_hazard_sr

name (Geographic Coordinate System),GCS_North_American_1983
factoryCode (WKID),4269
angularUnitName (Angular Unit),Degree
datumName (Datum),D_North_American_1983


In [14]:
# get datum transformation
flood_hazard_datum_transformation = get_datum_transformation(flood_hazard_sr, output_sr)
flood_hazard_datum_transformation

In [15]:
# Project the flood hazard layer
try:
    arcpy.Project_management(in_dataset = flood_hazard,
                                   out_dataset = flood_hazard_projected,
                                   out_coor_system = output_sr,
                                   transform_method = flood_hazard_datum_transformation)
    print("flood hazard reprojected successfully")
 
except arcpy.ExecuteError:
    print(arcpy.GetMessages(2))
except:
    print(Exception)

flood hazard reprojected successfully


In [15]:
# add occupancy field to the flood hazard layer and convert to raster
add_occupancy_field(flood_hazard_projected)
arcpy.conversion.FeatureToRaster(in_features = flood_hazard_projected,
                                 field = "occupancy",
                                 out_raster = flood_hazard_raster)

'occupancy' field already exists in FEMA_flood_hazard_zones_projected


<Result 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\FEMA_flood_hazard_zones.tif'>

#### Mosaic the technical threshold rasters

In [87]:
# create a list of rasters to mosaic
wind_rasters = [flood_hazard_raster, landscan_binary, wind_slope_binary]
solar_rasters = [flood_hazard_raster, landscan_binary, solar_slope_binary]

In [88]:
mosaic_workspace = os.path.join(mainInputFolder, "technicalExclusionRasters\\Mosaic")
mosaic_name_wind = "technicalExclusionMosaic_Wind.tif"
mosaic_name_solar = "technicalExclusionMosaic_Solar.tif"

In [90]:
# Perform the mosaic using MosaicToNewRaster_management function
arcpy.MosaicToNewRaster_management(solar_rasters, mosaic_workspace, mosaic_name_solar, pixel_type="1_BIT", number_of_bands=1)

<Result 'C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\technicalExclusionRasters\\Mosaic\\technicalExclusionMosaic_Solar.tif'>